In [8]:
import os
from typing import Literal
from tavily import TavilyClient
from langchain.agents import create_agent
from deepagents import create_deep_agent, SubAgent, CompiledSubAgent

# Using SubAgent

In [3]:
tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

def internet_search(
        query: str,
        max_results: int = 5,
        topic: Literal["general", "news", "finance"] = "general",
        include_raw_content: bool = False
):
    "Run a web search"
    return tavily_client.search(
        query=query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic
    )

research_subagent = {
    "name" : "research_agent",
    "description": "Used to research more in depth questions",
    "system_prompt": "You are great researcher",
    "tools": [internet_search],
    "model": "google_genai:gemini-2.5-flash",
}
subagets = [research_subagent]
agent = create_deep_agent(
    model="google_genai:gemini-2.5-flash",
    subagents=subagets
)

In [7]:
question = "What is llm?"
result = agent.invoke(
    {
        "messages" : [
            {
                "role" : "user",
                "content" : question
            }
        ]
    }
)
print(f"Result: {result["messages"][-1].text}")

Result: An LLM (Large Language Model) is a type of artificial intelligence program that is trained on a massive amount of text data. This allows it to understand, generate, and process human language in various ways. LLMs can perform tasks like:

*   **Text generation:** Writing articles, stories, poems, code, etc.
*   **Translation:** Translating text from one language to another.
*   **Summarization:** Condensing long texts into shorter summaries.
*   **Question answering:** Providing answers to questions based on its training data.
*   **Chatbots and conversational AI:** Engaging in human-like conversations.

They learn patterns, grammar, facts, and even some reasoning abilities from the vast datasets they are trained on, enabling them to produce coherent and contextually relevant responses.


# Using CompiledSubAgent

In [11]:
# Create a custom agent graph
custom_graph = create_agent(
    model="google_genai:gemini-2.5-flash",
    tools=[internet_search],
    system_prompt="You are a specialized agent for data analysis...")

# Use it as a custom subagent
custom_subagent = CompiledSubAgent(
    name="data-analyzer",
    description="Specialized agent for complex data analysis tasks",
    runnable=custom_graph,
)

subagets = [custom_subagent]

agent = create_deep_agent(
    model="google_genai:gemini-2.5-flash",
    tools=[internet_search],
    system_prompt="Give the answer for your best practises",
    subagents=subagets
    
)

In [12]:
question = "Give me information about Tavily!"
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content" : question
            }
        ]
    }
)

print(f"Result: \n {result["messages"][-1].content}")

Result: 
 Tavily is a company that provides a web access stack for AI agents, offering tools for real-time search, structured data extraction, and fully-rendered crawling. Their goal is to connect AI agents to the web in a way that is compliant with company-specific policies, enabling them to access and reason over live web data.

Key information about Tavily:

*   **Purpose:** To empower AI agents with the ability to access and utilize information from the live web.
*   **Offerings:** Search, Extract, and Crawl APIs designed for RAG (Retrieval Augmented Generation), autonomy, and production-grade agent systems.
*   **Funding:** They recently raised $20 million in Series A funding, led by Insight Partners.
*   **Founded:** 2024
*   **Company Size:** 11-50 employees
*   **Use Cases:** Powering conversational agents, enterprise AI data analysts, and other AI applications that require up-to-date, cleaned, and structured web content.
